# SBTi-Finance Tool - Portfolio Aggregation
In this notebook we'll give some examples on how the portfolio aggregation methods can be used.

Please see the [methodoloogy, guidance](https://sciencebasedtargets.org/wp-content/uploads/2020/07/Temperature-Scoring-Beta-Methodology.pdf) (Section 3.2) and the [technical documentation](http://getting-started.sbti-tool.org/) for more details on the different aggregation methods.

See 1_analysis_example for more in depth example of how to work with Jupyter Notebooks in general and SBTi notebooks in particular. 


## Setting up
First we will set up the imports, data providers, and load the portfolio. 

For more examples of this process, please refer to notebook 1 & 2 (analysis and quick calculation example).


In [ ]:
!pip install SBTi

In [1]:
%load_ext autoreload
%autoreload 2
import SBTi
from SBTi.data.excel import ExcelProvider
from SBTi.portfolio_aggregation import PortfolioAggregationMethod
from SBTi.portfolio_coverage_tvp import PortfolioCoverageTVP
from SBTi.temperature_score import TemperatureScore, Scenario, ScenarioType, EngagementType
from SBTi.target_validation import TargetProtocol
from SBTi.interfaces import ETimeFrames, EScope
%aimport -pandas
import pandas as pd

In [2]:
# Download the dummy data
import urllib.request
import os

if not os.path.isdir("data"):
    os.mkdir("data")
if not os.path.isfile("data/data_provider_example.xlsx"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/data/data_provider_example.xlsx", "data/data_provider_example.xlsx")
if not os.path.isfile("data/example_portfolio.csv"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/data/example_portfolio.csv", "data/example_portfolio.csv")

try:  # Import statement when run in Google Colabs
    from utils import  print_aggregations
except:  # Import statement when run locally
    from examples.utils import  print_aggregations

In [3]:
provider = ExcelProvider(path="data/data_provider_example.xlsx")
df_portfolio = pd.read_csv("data/example_portfolio.csv", encoding="iso-8859-1")
companies = SBTi.utils.dataframe_to_portfolio(df_portfolio)
scores_collection = {}

In [4]:
temperature_score = TemperatureScore(time_frames=list(SBTi.interfaces.ETimeFrames), scopes=[EScope.S1S2, EScope.S3, EScope.S1S2S3])
amended_portfolio = temperature_score.calculate(data_providers=[provider], portfolio=companies)

## Calculate the aggregated temperature score
Calculate an aggregated temperature score. This can be done using different aggregation methods. The termperature scores are calculated per time-frame/scope combination.

### WATS
Weighted Average Temperature Score (WATS): Temperature scores are allocated based on portfolio weights.
This method uses the "investment_value" field to be defined in your portfolio data.

In [5]:
temperature_score.aggregation_method = PortfolioAggregationMethod.WATS
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)
print_aggregations(aggregated_scores)
scores_collection.update({'WATS': aggregated_scores.mid.S1S2.all.score})

Timeframe  Scope      Temp score
short      S1S2       2.54
short      S3         2.93
short      S1S2S3     2.83
mid        S1S2       2.63
mid        S3         2.98
mid        S1S2S3     2.88
long       S1S2       3.16
long       S3         3.23
long       S1S2S3     3.21


### TETS
Total emissions weighted temperature score (TETS): Temperature scores are allocated based on historical emission weights using total company emissions. 
In addition to the portfolios "investment value" the TETS method requires company emissions, please refer to [Data Legends - Fundamental Data](https://ofbdabv.github.io/SBTi/Legends.html#fundamental-data) for more details

In [6]:
temperature_score.aggregation_method = PortfolioAggregationMethod.TETS
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)
print_aggregations(aggregated_scores)
scores_collection.update({'TEST': aggregated_scores.mid.S1S2.all.score})

Timeframe  Scope      Temp score
short      S1S2       2.84
short      S3         2.50
short      S1S2S3     2.62
mid        S1S2       2.92
mid        S3         2.76
mid        S1S2S3     2.82
long       S1S2       2.93
long       S3         3.21
long       S1S2S3     3.11


### MOTS
Market Owned emissions weighted temperature score (MOTS): Temperature scores are allocated based on an equity ownership approach.
In addition to the portfolios "investment value" the MOTS method requires company emissions and market cap, please refer to  [Data Legends - Fundamental Data](https://ofbdabv.github.io/SBTi/Legends.html#fundamental-data) for more details

In [7]:
temperature_score.aggregation_method = PortfolioAggregationMethod.MOTS
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)
print_aggregations(aggregated_scores)
scores_collection.update({'MOTS': aggregated_scores.mid.S1S2.all.score})

Timeframe  Scope      Temp score
short      S1S2       2.95
short      S3         2.66
short      S1S2S3     2.76
mid        S1S2       3.04
mid        S3         2.47
mid        S1S2S3     2.67
long       S1S2       3.13
long       S3         3.20
long       S1S2S3     3.18


### EOTS
Enterprise Owned emissions weighted temperature score (EOTS): Temperature scores are allocated based
on an enterprise ownership approach. 
In addition to the portfolios "investment value" the EOTS method requires company emissions and enterprise value, please refer to  [Data Legends - Fundamental Data](https://ofbdabv.github.io/SBTi/Legends.html#fundamental-data) for more details

In [8]:
temperature_score.aggregation_method = PortfolioAggregationMethod.EOTS
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)
print_aggregations(aggregated_scores)
scores_collection.update({'EOTS': aggregated_scores.mid.S1S2.all.score})

Timeframe  Scope      Temp score
short      S1S2       2.99
short      S3         2.50
short      S1S2S3     2.69
mid        S1S2       3.02
mid        S3         2.20
mid        S1S2S3     2.52
long       S1S2       3.16
long       S3         3.20
long       S1S2S3     3.19


### ECOTS
Enterprise Value + Cash emissions weighted temperature score (ECOTS): Temperature scores are allocated based on an enterprise value (EV) plus cash & equivalents ownership approach. 
In addition to the portfolios "investment value" the ECOTS method requires company emissions, company cash equivalents and enterprise value; please refer to  [Data Legends - Fundamental Data](https://ofbdabv.github.io/SBTi/Legends.html#fundamental-data) for more details

In [9]:
temperature_score.aggregation_method = PortfolioAggregationMethod.ECOTS
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)
print_aggregations(aggregated_scores)
scores_collection.update({'ECOTS': aggregated_scores.mid.S1S2.all.score})

Timeframe  Scope      Temp score
short      S1S2       2.96
short      S3         2.51
short      S1S2S3     2.68
mid        S1S2       3.00
mid        S3         2.41
mid        S1S2S3     2.63
long       S1S2       3.14
long       S3         3.21
long       S1S2S3     3.18


### AOTS
Total Assets emissions weighted temperature score (AOTS): Temperature scores are allocated based on a total assets ownership approach. 
In addition to the portfolios "investment value" the AOTS method requires company emissions and company total assets; please refer to  [Data Legends - Fundamental Data](https://ofbdabv.github.io/SBTi/Legends.html#fundamental-data) for more details

In [10]:
temperature_score.aggregation_method = PortfolioAggregationMethod.AOTS
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)
print_aggregations(aggregated_scores)
scores_collection.update({'AOTS': aggregated_scores.mid.S1S2.all.score})

Timeframe  Scope      Temp score
short      S1S2       2.01
short      S3         0.87
short      S1S2S3     1.22
mid        S1S2       2.09
mid        S3         3.11
mid        S1S2S3     2.80
long       S1S2       3.19
long       S3         3.21
long       S1S2S3     3.21


See below how each aggregation method impact the scores on timeframe: MID and scope: S1S2

In [11]:
print('Aggregation overview:')
for method, score in scores_collection.items():
    print('{} \t {:.2f}'.format(method, score))

Aggregation overview:
WATS 	 2.63
TEST 	 2.92
MOTS 	 3.04
EOTS 	 3.02
ECOTS 	 3.00
AOTS 	 2.09
